In [1]:
import pandas as pd

df = pd.read_csv('./flight_data_processed.csv')

print(df.tail())


     Departure Airport       Departure Time Arrival Airport  \
7504               DLI  2024-09-21 15:55:00             HAN   
7505               DLI  2024-09-21 07:00:00             HAN   
7506               DLI  2024-09-21 12:10:00             HAN   
7507               DLI  2024-09-21 13:05:00             VII   
7508               SGN  2024-09-21 15:10:00             VCS   

             Arrival Time Carrier Code  Flight Number  Price (VND) Duration  
7504  2024-09-21 17:50:00           QH           1422    3853000.0  PT1H55M  
7505  2024-09-21 08:50:00           W2           4404    3922000.0  PT1H50M  
7506  2024-09-21 14:00:00           W2           4412    3922000.0  PT1H50M  
7507  2024-09-21 14:40:00           A1           2712    5031000.0  PT1H35M  
7508  2024-09-21 16:10:00           VN           1885    9035000.0     PT1H  


In [2]:
df['Routine'] = df['Departure Airport'] + '-' + df['Arrival Airport']

df = df.drop(columns=['Flight Number', 'Duration', 'Departure Airport', 'Arrival Airport', 'Arrival Time'])

df.head()

,Departure Time,Carrier Code,Price (VND),Routine
0,2024-09-15 05:25:00,VJ,1541000.0,SGN-HAN
1,2024-09-15 06:00:00,VJ,1541000.0,SGN-HAN
2,2024-09-15 06:40:00,VJ,1541000.0,SGN-HAN
3,2024-09-15 07:00:00,VJ,1541000.0,SGN-HAN
4,2024-09-15 08:10:00,VJ,1541000.0,SGN-HAN


In [3]:
import numpy as np

num_parts = 50
df_parts = np.array_split(df, num_parts)


C:\Users\dinhg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [4]:
import mysql.connector
import os
from dotenv import load_dotenv

load_dotenv()

mydb = mysql.connector.connect(
  host=os.getenv('DB_CLOUD_HOST'),
  user=os.getenv('DB_CLOUD_USER'),
  password=os.getenv('DB_CLOUD_PASSWORD'),
  database=os.getenv('DB_CLOUD_NAME'),
  ssl_disabled=True,
  connection_timeout=1000000,
)

mycursor = mydb.cursor()

print("Đã kết nối thành công đến MySQL!")



Đã kết nối thành công đến MySQL!


In [5]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS flights (
    id INT AUTO_INCREMENT PRIMARY KEY,
    departure_time DATETIME,
    carrier_code VARCHAR(5),
    price DECIMAL(10, 2),
    routine VARCHAR(20)
)
"""

try:
    mycursor.execute(create_table_sql)
    print("Đang khởi tạo bảng flights")

    insert_sql = """
    INSERT INTO flights 
    (departure_time, carrier_code, price, routine) 
    VALUES (%s, %s, %s, %s)
    """

    total_inserted = 0
    for i, df_part in enumerate(df_parts):
        values = df_part[['Departure Time', 'Carrier Code', 'Price (VND)', 'Routine']].values.tolist()
        
        mycursor.executemany(insert_sql, values)
        mydb.commit()
        
        total_inserted += len(values)
        print(f"Đã chèn thành công {len(values)} bản ghi từ phần {i+1}/{len(df_parts)}. Tổng: {total_inserted}")

    print(f"Đã chèn thành công tổng cộng {total_inserted} bản ghi vào bảng flights.")
except mysql.connector.Error as error:
    print(f"Lỗi khi thao tác với MySQL: {error}")

Đang khởi tạo bảng flights
Đã chèn thành công 151 bản ghi từ phần 1/50. Tổng: 151
Đã chèn thành công 151 bản ghi từ phần 2/50. Tổng: 302
Đã chèn thành công 151 bản ghi từ phần 3/50. Tổng: 453
Đã chèn thành công 151 bản ghi từ phần 4/50. Tổng: 604
Đã chèn thành công 151 bản ghi từ phần 5/50. Tổng: 755
Đã chèn thành công 151 bản ghi từ phần 6/50. Tổng: 906
Đã chèn thành công 151 bản ghi từ phần 7/50. Tổng: 1057
Đã chèn thành công 151 bản ghi từ phần 8/50. Tổng: 1208
Đã chèn thành công 151 bản ghi từ phần 9/50. Tổng: 1359
Đã chèn thành công 150 bản ghi từ phần 10/50. Tổng: 1509
Đã chèn thành công 150 bản ghi từ phần 11/50. Tổng: 1659
Đã chèn thành công 150 bản ghi từ phần 12/50. Tổng: 1809
Đã chèn thành công 150 bản ghi từ phần 13/50. Tổng: 1959
Đã chèn thành công 150 bản ghi từ phần 14/50. Tổng: 2109
Đã chèn thành công 150 bản ghi từ phần 15/50. Tổng: 2259
Đã chèn thành công 150 bản ghi từ phần 16/50. Tổng: 2409
Đã chèn thành công 150 bản ghi từ phần 17/50. Tổng: 2559
Đã chèn thành công 

In [6]:
mycursor.execute("SELECT * FROM flights")

rows = mycursor.fetchall()

print("Dữ liệu từ bảng flights:")
for row in rows:
    print(row)


Dữ liệu từ bảng flights:
(1, datetime.datetime(2024, 9, 15, 5, 25), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(2, datetime.datetime(2024, 9, 15, 6, 0), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(3, datetime.datetime(2024, 9, 15, 6, 40), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(4, datetime.datetime(2024, 9, 15, 7, 0), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(5, datetime.datetime(2024, 9, 15, 8, 10), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(6, datetime.datetime(2024, 9, 15, 8, 45), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(7, datetime.datetime(2024, 9, 15, 10, 10), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(8, datetime.datetime(2024, 9, 15, 11, 0), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(9, datetime.datetime(2024, 9, 15, 11, 25), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(10, datetime.datetime(2024, 9, 15, 12, 0), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(11, datetime.datetime(2024, 9, 15, 12, 30), 'VJ', Decimal('1541000.00'), 'SGN-HAN')
(12, datetime.datetime(2024, 9, 15, 13, 20), 'VJ', Decimal(

In [7]:
if mydb.is_connected():
        mycursor.close()
        mydb.close()
        print("Kết nối MySQL đã đóng.")

Kết nối MySQL đã đóng.
